In [0]:
# %sql
# CREATE EXTERNAL LOCATION IF NOT EXISTS external_location_customer_stream_demo
#     URL 'abfss://containerstreamdemo@rajstorageparkargizmobox.dfs.core.windows.net/'
#     WITH (STORAGE CREDENTIAL storagecredentialgizmobox)
#     COMMENT 'External Location For the Gizmobox Data Lakehouse'

In [0]:
%sql
USE CATALOG newpavancatalog;
use schema bronze;

In [0]:
%sql
USE CATALOG newpavancatalog;
USE SCHEMA bronze;

CREATE EXTERNAL VOLUME IF NOT EXISTS volume_landing_Stream
    LOCATION 'abfss://landing@parkaru15sa.dfs.core.windows.net/customers';

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, TimestampType

customers_schema = StructType(fields=[StructField("customer_id", IntegerType()),
                                     StructField("customer_name", StringType()),
                                     StructField("date_of_birth", DateType()),
                                     StructField("telephone", StringType()),
                                     StructField("email", StringType()),
                                     StructField("member_since", DateType()),
                                     StructField("created_timestamp", TimestampType())
                                    ]
                              )

In [0]:
customers_df = (
                        spark.readStream
                         .format("json")
                         .schema(customers_schema)
                         .load("/Volumes/newpavancatalog/bronze/volume_landing_stream/"))

In [0]:
display(customers_df)

In [0]:
from pyspark.sql.functions import current_timestamp, col

customers_transformed_df = (customers_df.withColumn("file_path", col("_metadata.file_path"))
                                            .withColumn("ingestion_date", current_timestamp())
)

In [0]:
streaming_query = (
                    customers_transformed_df.writeStream
                        .format("delta")
                        .option("checkpointLocation", "/Volumes/newpavancatalog/bronze/volume_landing_stream/customers/")
                        .toTable("newpavancatalog.bronze.customer_stream")
)

In [0]:
streaming_query.stop()